# Data Processing & Feature Engineering

In [321]:
import pandas as pd
import pickle
import re
from tqdm import tqdm
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_score
import ast
from sklearn import preprocessing
from sklearn import tree
from sklearn.model_selection import GridSearchCV
import pydot
import graphviz
from sklearn.tree import export_graphviz
from scipy.stats.mstats import mode
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import recall_score

%matplotlib inline

In [322]:
# Read the pickle file

df = pd.read_pickle('yelp_lv_restaurants.pkl')

In [323]:
df

,city,latitude,type,attributes,address,review_count,postal_code,is_open,state,categories,business_id,stars,hours,longitude,name,neighborhood,index,is_restaurant
Order,,,,,,,,,,,,,,,,,,
37,Las Vegas,36.101,business,"[Alcohol: full_bar, BusinessAcceptsCreditCards...",4811 S Rainbow Blvd,15,89103,0,NV,"[Persian/Iranian, Restaurants, Ethnic Food, Fo...",saWZO6hB4B8P-mIzS1--Xw,2.5,None,-115.244,Kabob Palace,Spring Valley,16,1
71,Las Vegas,36.0991,business,"[Alcohol: none, Ambience: {'romantic': False, ...","4972 S Maryland Pkwy, Ste 22",33,89119,1,NV,"[Restaurants, Vegetarian, Indian]",hMh9XOwNQcu31NAOCqhAEw,3.5,None,-115.136,Taste of India,Southeast,16,1
72,Las Vegas,36.0658,business,"[Ambience: {'romantic': False, 'intimate': Fal...",2053 Pama Ln,35,89119,0,NV,"[American (New), Cafes, Restaurants]",pmJqSsCfgbo3TxPWpQNLIw,4.5,"[Wednesday 10:0-14:0, Thursday 10:0-14:0, Frid...",-115.124,Artisanal Foods Cafe,Southeast,16,1
102,Las Vegas,36.2385,business,"[Alcohol: full_bar, Ambience: {'romantic': Fal...",4341 N Rancho Dr,9,89130,0,NV,"[Nightlife, Bars, Restaurants, Thai, Sports Bars]",kUUBBLBHCasOl2a5nW9nAw,3.5,None,-115.232,Bailey's Sports Bar & Eatery,Northwest,16,1
103,Las Vegas,36.2497,business,"[Alcohol: none, Ambience: {'romantic': False, ...",4949 N Rancho Dr,41,89130,1,NV,"[Chicken Wings, Fast Food, Restaurants]",A2pZTpFXWC38z506XIhnBQ,3.5,"[Monday 10:30-0:0, Tuesday 10:30-0:0, Wednesda...",-115.245,Wingstop,Northwest,16,1
154,Las Vegas,36.1775,business,"[Alcohol: none, BusinessAcceptsCreditCards: Tr...",2230 W Bonanza Rd,6,89106,0,NV,"[Barbeque, Restaurants, Southern]",InDH4ZQ_byiQ5PyaqgHI8Q,2,None,-115.173,Big Mama's Soul Food Rib Shack,,16,1
177,Las Vegas,36.1591,business,"[Alcohol: beer_and_wine, BusinessAcceptsCredit...",7002 W Charleston Blvd,6,89145,0,NV,"[Restaurants, Burgers, Local Flavor, Barbeque]",WleVOQ9YhBYl4SWrlsLDhA,3.5,"[Monday 10:0-20:0, Tuesday 10:0-20:0, Wednesda...",-115.251,Red Apple Grill,Westside,16,1
179,Las Vegas,36.2973,business,"[Alcohol: none, Ambience: {'romantic': False, ...","7501 N Cimarron Rd, Ste 108",132,89131,1,NV,"[Desserts, Food, Pizza, Restaurants, Bakeries]",gMUAn6xcuE-TbY1seFw_Ww,4.5,"[Monday 11:0-21:0, Tuesday 11:0-20:0, Wednesda...",-115.27,Presto Calzone Bakery,Centennial,16,1
184,Las Vegas,36.2712,business,"[Alcohol: beer_and_wine, Ambience: {'romantic'...",7930 W Tropical Pkwy,74,89149,0,NV,"[Restaurants, Pizza]",kUntNQ5P9IrRzEoHdRxV-w,3.5,"[Monday 11:0-21:0, Tuesday 11:0-21:0, Wednesda...",-115.268,Mark Rich's New York Pizza & Pasta,Centennial,16,1


In [325]:
# Count how many LV restaurants are open vs closed

df['is_open'].value_counts()

1    3849
0    1582
Name: is_open, dtype: int64

In [326]:
# Replace null value in attributes with string 'none'

df['attributes'].fillna('none', inplace = True)

In [327]:
df['attributes'].isnull().sum()

0

In [328]:
# Create a list of attributes for all restaurants

def att_list_not_unique(data): 
    att_list= []
    for index, row in tqdm(data.iterrows()):
        for sub_list in row['attributes']:
            att = (sub_list.split(':')[0])
            att_list.append(att)
    return att_list

In [329]:
# Exam common attributes & save counter for top 18 attributes with more than 4000 occurance

common_att_counter = sorted(Counter(att_list_not_unique(df)).items(), key = lambda pair: pair[1], reverse = True)[:18]

5431it [00:00, 6610.99it/s]


In [330]:
common_att_list = list(zip(*common_att_counter))[0]

In [331]:
# Append all unique attributes of for all restaurants

def att_list(data): 
    att_list= []
    for index, row in tqdm(data.iterrows()):
        for sub_list in row['attributes']:
            att = (sub_list.split(':')[0])
            if att not in att_list:
                att_list.append(att)
    return att_list

In [332]:
att_list(df)

5431it [00:00, 7202.55it/s]


['Alcohol',
 'BusinessAcceptsCreditCards',
 'BusinessParking',
 'GoodForKids',
 'GoodForMeal',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'WheelchairAccessible',
 'Ambience',
 'BikeParking',
 'Caters',
 'HasTV',
 'NoiseLevel',
 'WiFi',
 'CoatCheck',
 'GoodForDancing',
 'HappyHour',
 'Music',
 'BusinessAcceptsBitcoin',
 'BYOBCorkage',
 'Corkage',
 'DriveThru',
 'BestNights',
 'Smoking',
 'BYOB',
 'DogsAllowed',
 'Open24Hours',
 'RestaurantsCounterService',
 'ByAppointmentOnly',
 'DietaryRestrictions',
 'n',
 'o',
 'e',
 'AgesAllowed',
 'HairSpecializesIn',
 'AcceptsInsurance']

In [333]:
# Create columns for all attributes 

def add_col(data, l):
    for item in l:
        data[item] = 'None'

In [334]:
add_col(df, common_att_list)

In [335]:
type(common_att_list)

tuple

In [336]:
# Fill in values for attributes

def fill_att(data, l):
    for index, row in tqdm(data.iterrows()):
        for item in l:
            result = re.findall(item +': (\w+)', str(row['attributes']))
            if not result:
                pass
            else:
                data.loc[index,item] = result[0]

In [337]:
fill_att(df, common_att_list)

5431it [01:11, 75.75it/s]


In [338]:
# Drop columns: city, type, state, index, address, BusinessAcceptsCreditCards, GoodForMeal, Ambience

df.drop('city', axis = 1, inplace = True)
df.drop('type', axis = 1, inplace = True)
df.drop('state', axis = 1, inplace = True)
df.drop('index', axis = 1, inplace = True)
df.drop('address', axis = 1, inplace = True)
df.drop('BusinessAcceptsCreditCards', axis = 1, inplace = True)
df.drop('GoodForMeal', axis = 1, inplace = True)
df.drop('Ambience', axis = 1, inplace = True)
df.drop('business_id', axis = 1, inplace = True)

In [339]:
# Fill restaurent price range with mode

df['RestaurantsPriceRange2'].replace('None', '2', inplace = True)

In [340]:
# Convert longitude, latitude, review_count, starts, price range into numeric

df[['latitude', 'longitude', 'review_count', 'stars', 'RestaurantsPriceRange2']] = df[['latitude', 'longitude', 'review_count', 'stars', 'RestaurantsPriceRange2']].apply(pd.to_numeric)

In [341]:
# Replace none with mode: RestaurantsGoodForGroups, GoodForKids, RestaurantsAttire, RestaurantsReservations, OutdoorSeating
#RestaurantsDelivery, alcohol (combine full bar and beer and wine), noiselevel(avg, quiet, loud (combine loud and very loud),)
# Wifi(no, free, paid), RestaurantsTableService

df['RestaurantsGoodForGroups'].replace('None', df['RestaurantsGoodForGroups'].mode()[0], inplace = True)
df['GoodForKids'].replace('None', df['GoodForKids'].mode()[0], inplace = True)
df['RestaurantsAttire'].replace('None', df['RestaurantsAttire'].mode()[0], inplace = True)
df['RestaurantsReservations'].replace('None', df['RestaurantsReservations'].mode()[0], inplace = True)
df['OutdoorSeating'].replace('None', df['OutdoorSeating'].mode()[0], inplace = True)
df['RestaurantsDelivery'].replace('None', df['RestaurantsDelivery'].mode()[0], inplace = True)
df['Alcohol'].replace('None', df['Alcohol'].mode()[0], inplace = True)
df['NoiseLevel'].replace('None', df['NoiseLevel'].mode()[0], inplace = True)
df['Alcohol'].replace('None', df['Alcohol'].mode()[0], inplace = True)
df['WiFi'].replace('None', df['WiFi'].mode()[0], inplace = True)
df['RestaurantsTableService'].replace('None', df['RestaurantsTableService'].mode()[0], inplace = True)

In [342]:
# Replace none with False: RestaurantsTakeOut, HasTV, BikeParking

df['RestaurantsTakeOut'].replace('None', 'False', inplace = True)
df['HasTV'].replace('None', 'False', inplace = True)
df['BikeParking'].replace('None', 'False', inplace = True)

In [343]:
# Categorize restaurents into downtown based on neighborhood (the strip or not the strip)

df['is_dt'] = 0
df.loc[df.neighborhood == 'The Strip', 'is_dt'] = 1
df['is_dt'].value_counts()

0    4613
1     818
Name: is_dt, dtype: int64

In [344]:
# Categorize alcohol into true or false

df.loc[df['Alcohol'] == 'full_bar', 'Alcohol'] = 'True'
df.loc[df['Alcohol'] == 'beer_and_wine', 'Alcohol'] = 'True'
df.loc[df['Alcohol'] == 'none', 'Alcohol'] = 'False'
df.loc[df['Alcohol'] == 'None', 'Alcohol'] = 'False'

df['Alcohol'].value_counts()

False    3120
True     2311
Name: Alcohol, dtype: int64

In [345]:
# Categorize attire into true(dressy or formal) and false(casual)

df.loc[df['RestaurantsAttire'] == 'casual', 'RestaurantsAttire'] = 'False'
df.loc[df['RestaurantsAttire'] == 'dressy', 'RestaurantsAttire'] = 'True'
df.loc[df['RestaurantsAttire'] == 'formal', 'RestaurantsAttire'] = 'True'

df['RestaurantsAttire'].value_counts()

False    5232
True      199
Name: RestaurantsAttire, dtype: int64

In [346]:
# Categorize noise level into average, quiet and loud
# Combine loud and very loud, and get dummies later

df.loc[df['NoiseLevel'] == 'very_loud', 'NoiseLevel'] = 'loud'

df['NoiseLevel'].value_counts()

average    4189
quiet       867
loud        375
Name: NoiseLevel, dtype: int64

In [347]:
# Create dummies for noise level - average, quiet and loud

df =  pd.concat([df, pd.get_dummies(df['NoiseLevel'])], axis = 1)

# Create dummies for WiFi - no, free, paid
df =  pd.concat([df, pd.get_dummies(df['WiFi'])], axis = 1)

In [348]:
# Rename dummy variable columns

df = df.rename(columns = {'average': 'noise_average', 'loud': 'noise_loud', 'loud': 'noise_loud', 'free': 'wifi_free', 'no': 'wifi_no', 'paid': 'wifi_paid'})

In [349]:
# Convert True and False into 1 and 0

df.replace(['True', 'False'], [1,0], inplace = True)

In [350]:
# DEAL WITH GOOD FOR MEAL
# Function to append 'good for meal' to dataframe as an indicator of what kind of meal the restaurent is good for

def meal_kind_only(data, meal):
    for index, row in tqdm(data.iterrows()):
        for sublist in row['attributes']:
            if sublist.find('GoodForMeal') != -1:  
                result = sublist.find(meal)
                if result == -1:
                    data.loc[index, meal] = 0
                else:
                    data.loc[index, meal] = 1

In [351]:
# dessert

meal_kind_only(df, "'dessert': True")

# latenight

meal_kind_only(df, "'latenight': True")

# lunch

meal_kind_only(df, "'lunch': True")

# dinner

meal_kind_only(df, "'dinner': True")

# breakfast

meal_kind_only(df, "'breakfast': True")

# brunch (to be EXCLUDED since it overlaps with breakfast)

meal_kind_only(df, "'brunch': True")

5431it [00:03, 1559.68it/s]
5431it [00:03, 1672.48it/s]
5431it [00:03, 1518.67it/s]
5431it [00:04, 1097.73it/s]
5431it [00:03, 1633.23it/s]
5431it [00:02, 1842.56it/s]


In [352]:
# Fill in the mode for those missing 'good for meal' (651 missing)

df["'dessert': True"].isnull().sum()

651

In [353]:
df["'dessert': True"].fillna(df["'dessert': True"].mode()[0], inplace = True)
df["'latenight': True"].fillna(df["'latenight': True"].mode()[0], inplace = True)
df["'lunch': True"].fillna(df["'lunch': True"].mode()[0], inplace = True)
df["'dinner': True"].fillna(df["'dinner': True"].mode()[0], inplace = True)
df["'breakfast': True"].fillna(df["'breakfast': True"].mode()[0], inplace = True)

In [354]:
df[["'dessert': True"]].isnull().sum()

'dessert': True    0
dtype: int64

In [355]:
df[["'latenight': True"]].isnull().sum()

'latenight': True    0
dtype: int64

In [356]:
df[["'lunch': True"]].isnull().sum()

'lunch': True    0
dtype: int64

In [357]:
df[["'dinner': True"]].isnull().sum()

'dinner': True    0
dtype: int64

In [358]:
df[["'breakfast': True"]].isnull().sum()

'breakfast': True    0
dtype: int64

In [359]:
df["'dessert': True"].value_counts()

0.0    5145
1.0     286
Name: 'dessert': True, dtype: int64

In [360]:
df["'latenight': True"].value_counts()

0.0    4900
1.0     531
Name: 'latenight': True, dtype: int64

In [361]:
df["'lunch': True"].value_counts()

0.0    3194
1.0    2237
Name: 'lunch': True, dtype: int64

In [362]:
df["'dinner': True"].value_counts()

0.0    3468
1.0    1963
Name: 'dinner': True, dtype: int64

In [363]:
df["'breakfast': True"].value_counts()

0.0    4847
1.0     584
Name: 'breakfast': True, dtype: int64

In [364]:
# DEAL WITH AMBIENCE
# Function to append 'good for meal' to dataframe as an indicator of what kind of meal the restaurent is good for

def ambience_kind_only(data, am):
    for index, row in tqdm(data.iterrows()):
        for sublist in row['attributes']:
            if sublist.find('Ambience') != -1:  
                result = sublist.find(am)
                if result == -1:
                    data.loc[index, am] = 0
                else:
                    data.loc[index, am] = 1

In [365]:
# ambience - romantic

ambience_kind_only(df, "'romantic': True")

# ambience - intimate

ambience_kind_only(df, "'intimate': True")

# ambience - classy

ambience_kind_only(df, "'classy': True")

# ambience - hipster

ambience_kind_only(df, "'hipster': True")

# ambience - divey

ambience_kind_only(df, "'divey': True")

# ambience - touristy

ambience_kind_only(df, "'touristy': True")

# ambience - upscale

ambience_kind_only(df, "'upscale': True")

# ambience - casual

ambience_kind_only(df, "'casual': True")

5431it [00:03, 1571.69it/s]
5431it [00:08, 664.23it/s]
5431it [00:07, 699.19it/s]
5431it [00:06, 881.33it/s] 
5431it [00:03, 1392.57it/s]
5431it [00:02, 2136.47it/s]
5431it [00:03, 1675.75it/s]
5431it [00:03, 1611.13it/s]


In [366]:
df["'romantic': True"].value_counts()

0.0    4432
1.0      61
Name: 'romantic': True, dtype: int64

In [367]:
df["'intimate': True"].value_counts()

0.0    4454
1.0      39
Name: 'intimate': True, dtype: int64

In [368]:
df["'classy': True"].value_counts()

0.0    4340
1.0     153
Name: 'classy': True, dtype: int64

In [369]:
df["'hipster': True"].value_counts()

0.0    4426
1.0      67
Name: 'hipster': True, dtype: int64

In [370]:
df["'divey': True"].value_counts()

0.0    4296
1.0     197
Name: 'divey': True, dtype: int64

In [371]:
df["'touristy': True"].value_counts()

0.0    4422
1.0      71
Name: 'touristy': True, dtype: int64

In [372]:
df["'upscale': True"].value_counts()

0.0    4417
1.0      76
Name: 'upscale': True, dtype: int64

In [373]:
df["'casual': True"].value_counts()

1.0    2410
0.0    2083
Name: 'casual': True, dtype: int64

In [374]:
# Fill in missing data for "'casual': True" - to be used later (938 missing)

df["'casual': True"].isnull().sum()

938

In [375]:
df["'casual': True"].fillna(df["'casual': True"].mode()[0], inplace = True)

In [376]:
df["'casual': True"].value_counts()

1.0    3348
0.0    2083
Name: 'casual': True, dtype: int64

In [377]:
df[["'casual': True"]].isnull().sum()

'casual': True    0
dtype: int64

In [378]:
# DEAL WITH CATEGORIES: 

categories_list = ([item for sublist in df['categories'] for item in sublist])

categories_list

['Persian/Iranian',
 'Restaurants',
 'Ethnic Food',
 'Food',
 'Greek',
 'Specialty Food',
 'Restaurants',
 'Vegetarian',
 'Indian',
 'American (New)',
 'Cafes',
 'Restaurants',
 'Nightlife',
 'Bars',
 'Restaurants',
 'Thai',
 'Sports Bars',
 'Chicken Wings',
 'Fast Food',
 'Restaurants',
 'Barbeque',
 'Restaurants',
 'Southern',
 'Restaurants',
 'Burgers',
 'Local Flavor',
 'Barbeque',
 'Desserts',
 'Food',
 'Pizza',
 'Restaurants',
 'Bakeries',
 'Restaurants',
 'Pizza',
 'Restaurants',
 'Chinese',
 'Restaurants',
 'Chinese',
 'Filipino',
 'Restaurants',
 'Mexican',
 'Restaurants',
 'Fast Food',
 'Restaurants',
 'Burgers',
 'Mexican',
 'Restaurants',
 'Fast Food',
 'Nightlife',
 'Food',
 'American (Traditional)',
 'Restaurants',
 'Sports Bars',
 'Burgers',
 'Bars',
 'Beer',
 'Wine & Spirits',
 'Food',
 'Vegetarian',
 'Vegan',
 'Restaurants',
 'American (New)',
 'Japanese',
 'Restaurants',
 'Restaurants',
 'Italian',
 'Restaurants',
 'American (Traditional)',
 'Persian/Iranian',
 'Food 

In [379]:
# Count the occurances of category keywords

top_cat_counter = sorted(Counter(categories_list).items(), key = lambda pair: pair[1], reverse = True)
top_cat_counter

[('Restaurants', 5431),
 ('Food', 1002),
 ('Fast Food', 805),
 ('Nightlife', 698),
 ('American (Traditional)', 681),
 ('Mexican', 680),
 ('Bars', 652),
 ('Sandwiches', 573),
 ('Pizza', 563),
 ('American (New)', 527),
 ('Burgers', 462),
 ('Chinese', 396),
 ('Italian', 358),
 ('Breakfast & Brunch', 339),
 ('Japanese', 310),
 ('Seafood', 271),
 ('Sushi Bars', 230),
 ('Asian Fusion', 223),
 ('Cafes', 211),
 ('Steakhouses', 208),
 ('Barbeque', 193),
 ('Chicken Wings', 190),
 ('Sports Bars', 166),
 ('Event Planning & Services', 164),
 ('Delis', 157),
 ('Coffee & Tea', 144),
 ('Desserts', 143),
 ('Thai', 136),
 ('Salad', 135),
 ('Arts & Entertainment', 126),
 ('Specialty Food', 123),
 ('Buffets', 115),
 ('Mediterranean', 113),
 ('Pubs', 112),
 ('Hawaiian', 109),
 ('Korean', 105),
 ('Caterers', 100),
 ('Bakeries', 100),
 ('Hot Dogs', 98),
 ('Diners', 95),
 ('Juice Bars & Smoothies', 94),
 ('Lounges', 93),
 ('Vegetarian', 92),
 ('Wine & Spirits', 85),
 ('Beer', 85),
 ('Vegan', 82),
 ('Vietnames

In [380]:
# Common categories - excluding restaurand and food as key word

common_cat_counter = list(zip(*top_cat_counter[2:16]))[0]

In [381]:
# Replace null value in attributes with string 'none'

df['categories'].fillna('none', inplace = True)

df['categories'].isnull().sum()

0

In [382]:
def fill_cat(data, l):
    for index, row in tqdm(data.iterrows()):
        for item in l:
            result = re.findall(item, str(row['categories']))
            if not result:
                data.loc[index,item] = 0
            else:
                data.loc[index,item] = 1

In [383]:
fill_cat(df, common_cat_counter)

5431it [00:39, 138.84it/s]


In [384]:
def fill_cat_american(data):
    for index, row in tqdm(data.iterrows()):
        result = re.findall('American', str(row['categories']))
        if not result:
            data.loc[index,'American'] = 0
        else:
            data.loc[index,'American'] = 1

In [385]:
fill_cat_american(df)

5431it [00:05, 1025.40it/s]


In [386]:
df.head()

,latitude,attributes,review_count,postal_code,is_open,categories,stars,hours,longitude,name,...,Sandwiches,Pizza,American (New),Burgers,Chinese,Italian,Breakfast & Brunch,Japanese,Seafood,American
Order,,,,,,,,,,,,,,,,,,,,,
37,36.101020,"[Alcohol: full_bar, BusinessAcceptsCreditCards...",15,89103,0,"[Persian/Iranian, Restaurants, Ethnic Food, Fo...",2.5,None,-115.244312,Kabob Palace,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,36.099142,"[Alcohol: none, Ambience: {'romantic': False, ...",33,89119,1,"[Restaurants, Vegetarian, Indian]",3.5,None,-115.136192,Taste of India,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,36.065839,"[Ambience: {'romantic': False, 'intimate': Fal...",35,89119,0,"[American (New), Cafes, Restaurants]",4.5,"[Wednesday 10:0-14:0, Thursday 10:0-14:0, Frid...",-115.123944,Artisanal Foods Cafe,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
102,36.238462,"[Alcohol: full_bar, Ambience: {'romantic': Fal...",9,89130,0,"[Nightlife, Bars, Restaurants, Thai, Sports Bars]",3.5,None,-115.231950,Bailey's Sports Bar & Eatery,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,36.249719,"[Alcohol: none, Ambience: {'romantic': False, ...",41,89130,1,"[Chicken Wings, Fast Food, Restaurants]",3.5,"[Monday 10:30-0:0, Tuesday 10:30-0:0, Wednesda...",-115.244528,Wingstop,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [387]:
df.drop('American (New)', axis = 1, inplace = True)
df.drop('American (Traditional)', axis = 1, inplace = True)

In [388]:
df['Fast Food'].value_counts()

0.0    4626
1.0     805
Name: Fast Food, dtype: int64

In [389]:
df['Nightlife'].value_counts()

0.0    4733
1.0     698
Name: Nightlife, dtype: int64

In [390]:
df['American'].value_counts()

0.0    4283
1.0    1148
Name: American, dtype: int64

In [391]:
df['Mexican'].value_counts()

0.0    4751
1.0     680
Name: Mexican, dtype: int64

In [392]:
df['Sandwiches'].value_counts()

0.0    4858
1.0     573
Name: Sandwiches, dtype: int64

In [393]:
df['Pizza'].value_counts()

0.0    4868
1.0     563
Name: Pizza, dtype: int64

In [394]:
df['Burgers'].value_counts()

0.0    4969
1.0     462
Name: Burgers, dtype: int64

In [395]:
df['Chinese'].value_counts()

0.0    5035
1.0     396
Name: Chinese, dtype: int64

In [396]:
df['Italian'].value_counts()

0.0    5073
1.0     358
Name: Italian, dtype: int64

In [397]:
df['Breakfast & Brunch'].value_counts()

0.0    5092
1.0     339
Name: Breakfast & Brunch, dtype: int64

In [398]:
df['Japanese'].value_counts()

0.0    5121
1.0     310
Name: Japanese, dtype: int64

In [399]:
df['Seafood'].value_counts()

0.0    5157
1.0     274
Name: Seafood, dtype: int64

In [400]:
df['RestaurantsAttire'].value_counts()

0    5232
1     199
Name: RestaurantsAttire, dtype: int64

In [401]:
# FINAL COLS TO INCLUDE: 
# "review_count", "stars", "RestaurantsPriceRange2", "RestaurantsPriceRange2",
# "RestaurantsGoodForGroups", "RestaurantsTakeOut", "GoodForKids", "RestaurantsAttire",
# "RestaurantsReservations", "OutdoorSeating", "RestaurantsDelivery", "Alcohol", "HasTV", 
# "RestaurantsTableService", "is_dt", "noise_loud", "quiet", "wifi_free", "wifi_paid", 
# "'dessert': True", "'latenight': True", "'lunch': True", "'dinner': True", 
# "'breakfast': True", "'casual': True", "Nightlife", "Bars", "Fast Food", "Burgers"
# "Pizza", "Chinese", "Mexican", "Japanese", "Italian", "Sandwiches", "Seafood", 
# "Breakfast & Brunch", "American"

# DEP VAR: is_open

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5431 entries, 37 to 144054
Data columns (total 61 columns):
latitude                    5431 non-null float64
attributes                  5431 non-null object
review_count                5431 non-null int64
postal_code                 5431 non-null object
is_open                     5431 non-null int64
categories                  5431 non-null object
stars                       5431 non-null float64
hours                       3873 non-null object
longitude                   5431 non-null float64
name                        5431 non-null object
neighborhood                5431 non-null object
is_restaurant               5431 non-null object
RestaurantsPriceRange2      5431 non-null int64
RestaurantsGoodForGroups    5431 non-null int64
RestaurantsTakeOut          5431 non-null int64
GoodForKids                 5431 non-null int64
RestaurantsAttire           5431 non-null int64
RestaurantsReservations     5431 non-null int64
OutdoorSeatin

# Building Classification Models

In [402]:
dfm = df

In [403]:
dfm.columns

Index(['latitude', 'attributes', 'review_count', 'postal_code', 'is_open',
       'categories', 'stars', 'hours', 'longitude', 'name', 'neighborhood',
       'is_restaurant', 'RestaurantsPriceRange2', 'RestaurantsGoodForGroups',
       'RestaurantsTakeOut', 'GoodForKids', 'RestaurantsAttire',
       'RestaurantsReservations', 'OutdoorSeating', 'RestaurantsDelivery',
       'BusinessParking', 'Alcohol', 'HasTV', 'RestaurantsTableService',
       'NoiseLevel', 'WiFi', 'BikeParking', 'is_dt', 'noise_average',
       'noise_loud', 'quiet', 'wifi_free', 'wifi_no', 'wifi_paid',
       ''dessert': True', ''latenight': True', ''lunch': True',
       ''dinner': True', ''breakfast': True', ''brunch': True',
       ''romantic': True', ''intimate': True', ''classy': True',
       ''hipster': True', ''divey': True', ''touristy': True',
       ''upscale': True', ''casual': True', 'Fast Food', 'Nightlife',
       'Mexican', 'Bars', 'Sandwiches', 'Pizza', 'Burgers', 'Chinese',
       'Italian', 'Break

In [404]:
# Scale numeric features - review_count, stars, estaurantsPriceRange2

dfm['review_countS'] = preprocessing.scale(dfm[['review_count']])
dfm['starsS'] = preprocessing.scale(dfm[['stars']])
dfm['RestaurantsPriceRange2S'] = preprocessing.scale(dfm[['RestaurantsPriceRange2']])

In [405]:
dfm.head()

,latitude,attributes,review_count,postal_code,is_open,categories,stars,hours,longitude,name,...,Burgers,Chinese,Italian,Breakfast & Brunch,Japanese,Seafood,American,review_countS,starsS,RestaurantsPriceRange2S
Order,,,,,,,,,,,,,,,,,,,,,
37,36.101020,"[Alcohol: full_bar, BusinessAcceptsCreditCards...",15,89103,0,"[Persian/Iranian, Restaurants, Ethnic Food, Fo...",2.5,None,-115.244312,Kabob Palace,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.384263,-1.199765,0.517435
71,36.099142,"[Alcohol: none, Ambience: {'romantic': False, ...",33,89119,1,"[Restaurants, Vegetarian, Indian]",3.5,None,-115.136192,Taste of India,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.329036,0.064849,0.517435
72,36.065839,"[Ambience: {'romantic': False, 'intimate': Fal...",35,89119,0,"[American (New), Cafes, Restaurants]",4.5,"[Wednesday 10:0-14:0, Thursday 10:0-14:0, Frid...",-115.123944,Artisanal Foods Cafe,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.322900,1.329463,0.517435
102,36.238462,"[Alcohol: full_bar, Ambience: {'romantic': Fal...",9,89130,0,"[Nightlife, Bars, Restaurants, Thai, Sports Bars]",3.5,None,-115.231950,Bailey's Sports Bar & Eatery,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.402671,0.064849,-0.975759
103,36.249719,"[Alcohol: none, Ambience: {'romantic': False, ...",41,89130,1,"[Chicken Wings, Fast Food, Restaurants]",3.5,"[Monday 10:30-0:0, Tuesday 10:30-0:0, Wednesda...",-115.244528,Wingstop,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.304491,0.064849,-0.975759


In [406]:
dfm['is_closed'] = 0

In [407]:
for index, row in tqdm(dfm.iterrows()):
    if row['is_open'] == 1:
        dfm.loc[index, 'is_closed'] = 0
    if row['is_open'] == 0:
        dfm.loc[index, 'is_closed'] = 1
    

5431it [00:04, 1191.07it/s]


In [408]:
dfm.is_closed.value_counts()

0    3849
1    1582
Name: is_closed, dtype: int64

In [409]:
dfm.is_open.value_counts()

1    3849
0    1582
Name: is_open, dtype: int64

In [410]:
Y = dfm['is_open']
X = dfm[["review_count", "stars", "RestaurantsPriceRange2S",\
"RestaurantsGoodForGroups", "RestaurantsTakeOut", "GoodForKids", "RestaurantsAttire",\
"RestaurantsReservations", "OutdoorSeating", "RestaurantsDelivery", "Alcohol", "HasTV",\
 "is_dt", "noise_loud", "quiet", "wifi_free", "wifi_paid",\
"'dessert': True", "'latenight': True", "'lunch': True", "'dinner': True",\
"'breakfast': True", "'casual': True", "Nightlife", "Bars", "Fast Food", "Burgers",\
"Pizza", "Chinese", "Mexican", "Japanese", "Italian", "Sandwiches", "Seafood",\
"Breakfast & Brunch", "American"]]

In [411]:
Y.value_counts()

1    3849
0    1582
Name: is_open, dtype: int64

In [413]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 4444)

In [414]:
# Scale data set fror decision tree

stdsc = preprocessing.StandardScaler()

X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.fit_transform(X_test)

In [415]:
X_train_std

array([[-0.30004391,  0.07087717,  0.51670665, ...,  4.27719724,
        -0.25495922, -0.51631173],
       [-0.19604554,  0.07087717,  0.51670665, ..., -0.23379796,
        -0.25495922, -0.51631173],
       [-0.20198831,  0.70476923,  0.51670665, ..., -0.23379796,
        -0.25495922,  1.93681439],
       ..., 
       [ 0.47845816, -0.56301489,  0.51670665, ..., -0.23379796,
         3.9221959 , -0.51631173],
       [-0.08907579,  1.33866129, -0.98482392, ..., -0.23379796,
        -0.25495922, -0.51631173],
       [-0.33570049,  1.33866129,  0.51670665, ..., -0.23379796,
        -0.25495922, -0.51631173]])

In [416]:
X.corr()

,review_count,stars,RestaurantsPriceRange2S,RestaurantsGoodForGroups,RestaurantsTakeOut,GoodForKids,RestaurantsAttire,RestaurantsReservations,OutdoorSeating,RestaurantsDelivery,...,Burgers,Pizza,Chinese,Mexican,Japanese,Italian,Sandwiches,Seafood,Breakfast & Brunch,American
review_count,1.000000,0.162408,0.240400,0.056920,-0.061517,-0.090154,0.118132,0.203451,0.073367,-0.070240,...,0.006394,-0.052370,-0.023276,-0.052821,0.058531,0.022033,-0.039569,0.059520,0.142899,0.093439
stars,0.162408,1.000000,0.106912,0.134141,-0.010958,-0.051903,0.090727,0.186904,0.118382,0.087823,...,-0.147990,-0.080689,-0.056148,0.026996,0.107291,0.029426,-0.026238,0.067077,0.050902,0.030437
RestaurantsPriceRange2S,0.240400,0.106912,1.000000,0.048193,-0.373879,-0.411550,0.482854,0.499509,0.004554,-0.082040,...,-0.107325,-0.006227,-0.049463,-0.177249,0.119013,0.134132,-0.140009,0.189615,0.007357,0.137249
RestaurantsGoodForGroups,0.056920,0.134141,0.048193,1.000000,-0.038227,0.027010,0.019966,0.136435,0.042560,-0.036458,...,0.033506,-0.104649,0.028463,0.018783,0.034484,0.007603,-0.069633,0.011828,0.027394,0.058109
RestaurantsTakeOut,-0.061517,-0.010958,-0.373879,-0.038227,1.000000,0.261645,-0.307407,-0.090902,0.102809,0.163129,...,0.085145,0.048302,0.053812,0.084465,0.015916,-0.025787,0.097179,-0.040453,0.043684,-0.035384
GoodForKids,-0.090154,-0.051903,-0.411550,0.027010,0.261645,1.000000,-0.387605,-0.260205,-0.008881,0.125223,...,0.059495,0.060557,0.075125,0.102105,0.006713,-0.048173,0.099087,-0.070938,0.028510,-0.218074
RestaurantsAttire,0.118132,0.090727,0.482854,0.019966,-0.307407,-0.387605,1.000000,0.259800,-0.003407,-0.076218,...,-0.055955,-0.063109,-0.009461,-0.070821,0.019606,0.078526,-0.054219,0.134150,-0.030065,0.050253
RestaurantsReservations,0.203451,0.186904,0.499509,0.136435,-0.090902,-0.260205,0.259800,1.000000,0.093982,0.027684,...,-0.103889,-0.051323,0.036728,-0.084632,0.167142,0.157535,-0.158514,0.165492,-0.037394,0.048288
OutdoorSeating,0.073367,0.118382,0.004554,0.042560,0.102809,-0.008881,-0.003407,0.093982,1.000000,0.028647,...,0.013854,-0.005659,-0.095849,0.034549,-0.058961,0.075432,0.064419,-0.015878,0.099780,0.059909
RestaurantsDelivery,-0.070240,0.087823,-0.082040,-0.036458,0.163129,0.125223,-0.076218,0.027684,0.028647,1.000000,...,-0.084103,0.326741,0.111747,-0.105002,-0.057350,0.113547,0.069649,-0.026839,-0.030533,-0.124253


### Dummy Predictor

In [417]:
DM = DummyClassifier(strategy='stratified', random_state=4444)

DM.fit(X_train_std, Y_train)
pred = DM.predict(X_test_std)
score = accuracy_score(Y_test, pred)
score

0.60613496932515343

In [418]:
DM = DummyClassifier(strategy='stratified', random_state=4444)

DM.fit(X_train_std, Y_train)
pred = DM.predict(X_test_std)
score = recall_score(Y_test, pred)
score

0.73961937716262971

### Logistic Regression

In [419]:
# Out of Sample Using Default Setting - Accuracy

LR = LogisticRegression('l1')

LR.fit(X_train_std, Y_train)
pred = LR.predict(X_test_std)
score = accuracy_score(Y_test, pred)
score

0.74355828220858899

In [420]:
# In Sample Using Default Setting - Accuracy

LR = LogisticRegression('l1')

LR.fit(X_train_std, Y_train)

pred = LR.predict(X_train_std)
score = accuracy_score(Y_train, pred)
score

0.74769797421731121

In [421]:
# ASK ABOUT HOW TO INTERPRET THE COEFF AND WHICH ONES TO DROP
# Dropped RestaurantsTableServices

X.columns

Index(['review_count', 'stars', 'RestaurantsPriceRange2S',
       'RestaurantsGoodForGroups', 'RestaurantsTakeOut', 'GoodForKids',
       'RestaurantsAttire', 'RestaurantsReservations', 'OutdoorSeating',
       'RestaurantsDelivery', 'Alcohol', 'HasTV', 'is_dt', 'noise_loud',
       'quiet', 'wifi_free', 'wifi_paid', ''dessert': True',
       ''latenight': True', ''lunch': True', ''dinner': True',
       ''breakfast': True', ''casual': True', 'Nightlife', 'Bars', 'Fast Food',
       'Burgers', 'Pizza', 'Chinese', 'Mexican', 'Japanese', 'Italian',
       'Sandwiches', 'Seafood', 'Breakfast & Brunch', 'American'],
      dtype='object')

In [422]:
# GridSearchCV - Logistic Regression - with L1 penalty - Accuracy

# define the parameter values that should be searched

LR_1 = LogisticRegression(penalty='l1')

param_grid_LR_1 = {"solver": ["liblinear"]}

# create a parameter grid: map the parameter names to the values that should be searched
print(param_grid_LR_1)

# instantiate the grid
grid_LR_1 = GridSearchCV(LR_1, param_grid_LR_1, cv=10, scoring='accuracy')
grid_LR_1.fit(X_train_std, Y_train)
print (grid_LR_1.grid_scores_)
print (grid_LR_1.best_estimator_)

{'solver': ['liblinear']}
[mean: 0.74533, std: 0.01385, params: {'solver': 'liblinear'}]
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


//anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [461]:
# Create a list of features and coefs
i = []
for n in list(range(36)):
    i.append(X.columns[n])
    i.append(grid_LR_1.best_estimator_.coef_[0][n])

In [462]:
i

['review_count',
 1.6796259633553636,
 'stars',
 0.016046647841766223,
 'RestaurantsPriceRange2S',
 -0.0014136174164024096,
 'RestaurantsGoodForGroups',
 -0.083467014425517086,
 'RestaurantsTakeOut',
 0.065855273898838426,
 'GoodForKids',
 -0.038559377744888455,
 'RestaurantsAttire',
 -0.10130848270734759,
 'RestaurantsReservations',
 -0.24019537125987675,
 'OutdoorSeating',
 -0.12651453293793954,
 'RestaurantsDelivery',
 -0.025117876085646765,
 'Alcohol',
 -0.15532795290311166,
 'HasTV',
 0.029384740555112795,
 'is_dt',
 0.028278272264585916,
 'noise_loud',
 0.011457301706683194,
 'quiet',
 -0.086452768420441709,
 'wifi_free',
 0.20254911100867978,
 'wifi_paid',
 0.009279141818524626,
 "'dessert': True",
 0.15070107352726309,
 "'latenight': True",
 0.078410448220778264,
 "'lunch': True",
 0.070398532275563225,
 "'dinner': True",
 0.081619680299880065,
 "'breakfast': True",
 0.12530781487762088,
 "'casual': True",
 -0.17424131573673546,
 'Nightlife',
 0.21342636330047504,
 'Bars',
 0.0

In [458]:
# Coef for the param tuned logit reg

grid_LR_1.best_estimator_.coef_[0][1]

0.016046647841766223

In [424]:
# OUT OF SAMPLE ACCURACY FOR BEST LOGIT L1 MODEL

pred_OLR = grid_LR_1.best_estimator_.predict(X_test_std)
score = accuracy_score(Y_test, pred_OLR)
score

0.74355828220858899

In [474]:
# In Sample Using Default Setting - Accuracy for L2

LR = LogisticRegression('l2')

LR.fit(X_train_std, Y_train)

pred = LR.predict(X_train_std)
score = accuracy_score(Y_train, pred)
score

0.7471717968955538

In [425]:
# GridSearchCV - Logistic Regression - with L2 penalty

# define the parameter values that should be searched

LR_2 = LogisticRegression(penalty='l2')

param_grid_LR_2 = {"solver": ["newton-cg", "lbfgs", "liblinear", "sag"]}

# create a parameter grid: map the parameter names to the values that should be searched
print(param_grid_LR_2)

# instantiate the grid
grid_LR_2 = GridSearchCV(LR_2, param_grid_LR_2, cv=10, scoring='accuracy')
grid_LR_2.fit(X_train_std, Y_train)

grid_LR_2.grid_scores_

{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag']}


//anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.74401, std: 0.01380, params: {'solver': 'newton-cg'},
 mean: 0.74401, std: 0.01380, params: {'solver': 'lbfgs'},
 mean: 0.74401, std: 0.01380, params: {'solver': 'liblinear'},
 mean: 0.74401, std: 0.01380, params: {'solver': 'sag'}]

In [426]:
grid_LR_2.best_estimator_.coef_

array([[ 1.66435813,  0.01912561, -0.00193546, -0.08512675,  0.06622729,
        -0.04124232, -0.1033214 , -0.24063271, -0.12823563, -0.027567  ,
        -0.15875221,  0.03116689,  0.0316517 ,  0.01236108, -0.08853683,
         0.20431156,  0.01057499,  0.15294654,  0.07943886,  0.07202636,
         0.08711849,  0.12690873, -0.17622797,  0.21235109,  0.02502965,
         0.55767186, -0.04876397,  0.17606073,  0.0682865 ,  0.15669802,
        -0.06765057, -0.09430475,  0.16905117,  0.0546751 ,  0.03127055,
         0.00580291]])

### KNN model

In [427]:
# Out of Sample Using Default Setting

KNN = KNeighborsClassifier()

KNN.fit(X_train_std, Y_train)

pred = KNN.predict(X_test_std)
score = accuracy_score(Y_test, pred)
score

0.70490797546012274

In [428]:
# In Sample Using Default Setting

KNN = KNeighborsClassifier()

KNN.fit(X_train_std, Y_train)

pred = KNN.predict(X_train_std)
score = accuracy_score(Y_train, pred)
score

0.79952644041041832

In [429]:
# GridSearchCV

# define the parameter values that should be searched
k_range_KNN = list(range(1, 51))
print(k_range_KNN)

# create a parameter grid: map the parameter names to the values that should be searched
param_grid_KNN = dict(n_neighbors=k_range_KNN)
print(param_grid_KNN)

# instantiate the grid
grid_KNN = GridSearchCV(KNN, param_grid_KNN, cv=10, scoring='accuracy')
grid_KNN.fit(X_train_std, Y_train)

sorted(grid_KNN.grid_scores_, key = lambda x: x[1], reverse = True)[:3]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]}


//anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.72902, std: 0.01096, params: {'n_neighbors': 45},
 mean: 0.72823, std: 0.00820, params: {'n_neighbors': 46},
 mean: 0.72744, std: 0.01156, params: {'n_neighbors': 39}]

In [430]:
# OUT OF SAMPLE ACCURACY FOR BEST KNN MODEL

pred_KNN = grid_KNN.best_estimator_.predict(X_test_std)
score = accuracy_score(Y_test, pred_KNN)
score

0.73067484662576687

### Gaussian Naive Bayes model

In [431]:
# Out of Sample Using Default Setting

NB = GaussianNB()

NB.fit(X_train_std, Y_train)

pred = NB.predict(X_test_std)
score = accuracy_score(Y_test, pred)
score

0.61349693251533743

In [432]:
# In Sample Using Default Setting

NB = GaussianNB()

NB.fit(X_train_std, Y_train)

pred = NB.predict(X_train_std)
score = accuracy_score(Y_train, pred)
score

0.61168113654301504

In [433]:
# GridSearchCV - NO PARAMETER TUNING

### Bernoulli Naive Bayes model

In [434]:
# Out of Sample Using Default Setting

BNB = BernoulliNB()

BNB.fit(X_train_std, Y_train)

pred = BNB.predict(X_test_std)
score = accuracy_score(Y_test, pred)
score

0.71840490797546008

In [435]:
# In Sample Using Default Setting

BNB = BernoulliNB()

BNB.fit(X_train_std, Y_train)

pred = BNB.predict(X_train_std)
score = accuracy_score(Y_train, pred)
score

0.7250723493817417

In [436]:
# GridSearchCV - Bernoulli Naive Bayes

# define the parameter values that should be searched

BNB = BernoulliNB()

param_grid_BNB = {"alpha": [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}

# create a parameter grid: map the parameter names to the values that should be searched
print(param_grid_BNB)

# instantiate the grid
grid_BNB = GridSearchCV(BNB, param_grid_BNB, cv=10, scoring='accuracy')
grid_BNB.fit(X_train_std, Y_train)

sorted(grid_BNB.grid_scores_, key = lambda x: x[1], reverse = True)[:5]

{'alpha': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}


//anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.72192, std: 0.01346, params: {'alpha': 0.1},
 mean: 0.72165, std: 0.01321, params: {'alpha': 0.2},
 mean: 0.72139, std: 0.01369, params: {'alpha': 0},
 mean: 0.72139, std: 0.01338, params: {'alpha': 0.3},
 mean: 0.72139, std: 0.01260, params: {'alpha': 0.4}]

In [437]:
# OUT OF SAMPLE ACCURACY FOR BEST BNB MODEL

pred_BNB = grid_BNB.best_estimator_.predict(X_test_std)
score = accuracy_score(Y_test, pred_BNB)
score

0.71840490797546008

### Support Vector Machine model

In [438]:
# Out of Sample with default setting

SVM = SVC()

SVM.fit(X_train_std, Y_train)

pred = SVM.predict(X_test_std)
score = accuracy_score(Y_test, pred)
score

0.73987730061349688

In [439]:
# In Sample with default setting

SVM = SVC()

SVM.fit(X_train_std, Y_train)

pred = SVM.predict(X_train_std)
score = accuracy_score(Y_train, pred)
score

0.80347277032359909

In [440]:
# GridSearchCV - SVC

# define the parameter values that should be searched

SVM = SVC()

param_grid_SVM = {"C": [1, 10, 100],
                 "kernel": ["linear", "poly", "rbf", "sigmoid"]
                 }

# create a parameter grid: map the parameter names to the values that should be searched
print(param_grid_SVM)

# instantiate the grid
grid_SVM = GridSearchCV(SVM, param_grid_SVM, cv=10, scoring='accuracy')
grid_SVM.fit(X_train_std, Y_train)

sorted(grid_SVM.grid_scores_, key = lambda x: x[1], reverse = True)[:5]

{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 10, 100]}


//anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.75164, std: 0.01410, params: {'kernel': 'rbf', 'C': 1},
 mean: 0.72954, std: 0.01065, params: {'kernel': 'linear', 'C': 10},
 mean: 0.72902, std: 0.01746, params: {'kernel': 'rbf', 'C': 10},
 mean: 0.72823, std: 0.01089, params: {'kernel': 'linear', 'C': 100},
 mean: 0.72770, std: 0.00828, params: {'kernel': 'linear', 'C': 1}]

In [441]:
# OUT OF SAMPLE ACCURACY FOR BEST SVM MODEL

pred_SVM = grid_SVM.best_estimator_.predict(X_test_std)
score = accuracy_score(Y_test, pred_SVM)
score

0.73987730061349688

### Decision Tree Model

In [442]:
# Dataset for decision tree - want to include all dummy vairables

Y_dt = dfm['is_open']

X_dt = dfm[["review_count", "stars", "RestaurantsPriceRange2",\
"RestaurantsGoodForGroups", "RestaurantsTakeOut", "GoodForKids", "RestaurantsAttire",\
"RestaurantsReservations", "OutdoorSeating", "RestaurantsDelivery", "Alcohol", "HasTV",\
"RestaurantsTableService", "is_dt", "noise_average", "noise_loud", "quiet", "wifi_no", "wifi_free", "wifi_paid",\
"'dessert': True", "'latenight': True", "'lunch': True", "'dinner': True",\
"'breakfast': True", "'casual': True", "Nightlife", "Bars", "Fast Food", "Burgers",\
"Pizza", "Chinese", "Mexican", "Japanese", "Italian", "Sandwiches", "Seafood",\
"Breakfast & Brunch", "American"]]

In [443]:
X_train_dt, X_test_dt, Y_train_dt, Y_test_dt = train_test_split(X_dt, Y_dt, test_size = 0.3, random_state = 4444)

In [444]:
# Scale data set for decision tree

stdsc = preprocessing.StandardScaler()

X_train_dt_std = stdsc.fit_transform(X_train_dt)
X_test_dt_std = stdsc.fit_transform(X_test_dt)

#### Decision Tree model with Gini

In [445]:
# Out of Sample

DT = DecisionTreeClassifier()

DT.fit(X_train_dt_std, Y_train)

pred = DT.predict(X_test_dt_std)
score = accuracy_score(Y_test, pred)
score

0.65460122699386503

In [446]:
# In Sample

DT = DecisionTreeClassifier()

DT.fit(X_train_dt_std, Y_train)

pred = DT.predict(X_train_dt_std)
score = accuracy_score(Y_train, pred)
score

0.99973691133912124

In [447]:
# GridSearchCV - Decision Tree

# define the parameter values that should be searched

DT = DecisionTreeClassifier()

param_grid_DT = {"criterion": ["gini", "entropy"],
                "max_depth": [3,4,5,6,7,8,9,10, None],
                "min_samples_leaf": [1,2,3,4,5],
                "class_weight": ["balanced"]}

# create a parameter grid: map the parameter names to the values that should be searched
print(param_grid_DT)

# instantiate the grid
grid_DT = GridSearchCV(DT, param_grid_DT, cv=10, scoring='accuracy')
grid_DT.fit(X_train_dt_std, Y_train)

sorted(grid_DT.grid_scores_, key = lambda x: x[1], reverse = True)[:5]

{'criterion': ['gini', 'entropy'], 'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None], 'min_samples_leaf': [1, 2, 3, 4, 5], 'class_weight': ['balanced']}


//anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.71534, std: 0.01516, params: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'class_weight': 'balanced'},
 mean: 0.71534, std: 0.01209, params: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'class_weight': 'balanced'},
 mean: 0.69113, std: 0.02024, params: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'class_weight': 'balanced'},
 mean: 0.68640, std: 0.01971, params: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 4, 'class_weight': 'balanced'},
 mean: 0.68587, std: 0.01763, params: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 4, 'class_weight': 'balanced'}]

In [448]:
# OUT OF SAMPLE ACCURACY FOR BEST DECISION TREE MODEL

pred_DT = grid_DT.best_estimator_.predict(X_test_dt)
score = accuracy_score(Y_test_dt, pred_DT)
score

0.55950920245398772

####  Random Forest Model with Gini

In [464]:
# Out of Sample

RF = RandomForestClassifier()

RF.fit(X_train_dt_std, Y_train)

pred = RF.predict(X_test_dt_std)
score = accuracy_score(Y_test, pred)
score

0.71717791411042942

In [465]:
# In Sample

RF = RandomForestClassifier()

RF.fit(X_train_dt_std, Y_train)

pred = RF.predict(X_train_dt_std)
score = accuracy_score(Y_train, pred)
score

0.98973954222573002

Overfitting under default since in sample accuracy is much higher than out of sample

####  Random Forest Model with Entropy

In [466]:
# Out of Sample

RF = RandomForestClassifier(criterion='entropy')

RF.fit(X_train_dt_std, Y_train)

pred = RF.predict(X_test_dt_std)
score = accuracy_score(Y_test, pred)
score

0.73067484662576687

In [467]:
# In Sample

RF = RandomForestClassifier(criterion='entropy')

RF.fit(X_train_dt_std, Y_train)

pred = RF.predict(X_train_dt_std)
score = accuracy_score(Y_train, pred)
score

0.98526703499079193

In [468]:
# GridSearch - Random Forest

RF = RandomForestClassifier()

param_grid_RF = {"n_estimators": [5, 10, 20, 100],
                "criterion": ["gini", "entropy"],
                "max_depth": [3,4,5,6,7,8,9,10, None],
                "min_samples_leaf": [1,2,3,4,5],
                "class_weight": ["balanced"]}

# create a parameter grid: map the parameter names to the values that should be searched
print(param_grid_RF)

# instantiate the grid
grid_RF = GridSearchCV(RF, param_grid_RF, cv=10, scoring='accuracy')
grid_RF.fit(X_train_dt_std, Y_train)

sorted(grid_RF.grid_scores_, key = lambda x: x[1], reverse = True)[:5]

{'criterion': ['gini', 'entropy'], 'n_estimators': [5, 10, 20, 100], 'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None], 'min_samples_leaf': [1, 2, 3, 4, 5], 'class_weight': ['balanced']}


//anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.76585, std: 0.01240, params: {'criterion': 'gini', 'n_estimators': 100, 'max_depth': None, 'min_samples_leaf': 1, 'class_weight': 'balanced'},
 mean: 0.76427, std: 0.01418, params: {'criterion': 'entropy', 'n_estimators': 100, 'max_depth': None, 'min_samples_leaf': 1, 'class_weight': 'balanced'},
 mean: 0.76190, std: 0.01813, params: {'criterion': 'gini', 'n_estimators': 100, 'max_depth': None, 'min_samples_leaf': 2, 'class_weight': 'balanced'},
 mean: 0.76112, std: 0.00720, params: {'criterion': 'entropy', 'n_estimators': 20, 'max_depth': None, 'min_samples_leaf': 1, 'class_weight': 'balanced'},
 mean: 0.76059, std: 0.01684, params: {'criterion': 'gini', 'n_estimators': 20, 'max_depth': None, 'min_samples_leaf': 1, 'class_weight': 'balanced'}]

In [469]:
# OUT OF SAMPLE ACCURACY FOR BEST RANDOM FOREST MODEL

pred_RF = pred_RF = grid_RF.best_estimator_.predict(X_test_dt)
score = accuracy_score(Y_test_dt, pred_RF)
score

0.70920245398773007

### Ensemble - Voting Classifier 

After tuning parameters, logistic (74.3% accuracy), SVM (73.98%) and KNN (73.06%) generated the best out of sample performance using the test set. I will apply ensemble techniques by using a voting classifier of these three.

In [470]:
clf1 = grid_LR_1.best_estimator_
clf2 = grid_SVM.best_estimator_
clf3 = grid_KNN.best_estimator_

In [471]:
vclf = VotingClassifier(estimators=[('LR1', clf1), ('SVM', clf2), ('KNN', clf3)], voting='hard')

vclf.fit(X_train_std, Y_train)

pred_vote = vclf.predict(X_test_std)

score = accuracy_score(Y_test, pred_vote)
print (score)

recall = recall_score(Y_test, pred_vote)
print (recall)

0.747239263804
0.952422145329


#### Voting classifier improved the accuracy to 74.7% (compared to 60%) and recall to 95% (compared to 74%)!

In [472]:
# Checking voting accuracy for training set

for clf, label in zip([clf1, clf2, clf3, vclf], ['Logistic Regression', 'SVM', 'KNN', 'Ensemble']):
    scores = cross_val_score(clf, X_train_std, Y_train, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.74 (+/- 0.01) [Logistic Regression]
Accuracy: 0.75 (+/- 0.01) [SVM]
Accuracy: 0.73 (+/- 0.01) [KNN]
Accuracy: 0.75 (+/- 0.01) [Ensemble]


In [473]:
# Checking voting accuracy for testing set

for clf, label in zip([clf1, clf2, clf3, vclf], ['Logistic Regression', 'SVM', 'KNN', 'Ensemble']):
    scores = cross_val_score(clf, X_test_std, Y_test, cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.73 (+/- 0.03) [Logistic Regression]
Accuracy: 0.72 (+/- 0.02) [SVM]
Accuracy: 0.71 (+/- 0.01) [KNN]
Accuracy: 0.72 (+/- 0.02) [Ensemble]


### Example Prediction using Voting Classification

In [480]:
# Scale data set for example pred

stdsc = preprocessing.StandardScaler()

X_ex = stdsc.fit_transform(X)

In [505]:
# Example 1: Bailey's Sports Bar & Eatery (closed)

ex1 = X_ex[3]

vclf.predict(ex1)

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([1])

In [501]:
# Example  2 - Presto Calzone Bakery (open) 
ex2 = X_ex[7]

vclf.predict(ex2)

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([1])

In [509]:
# Example 3 - Kabob Palace (closed)
ex3 = X_ex[0]

vclf.predict(ex3)

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([0])

# Flask Web App - Work in Progress

In [ ]:
PREDICTOR = vclf.fit(X_train_std, Y_train)

In [3]:
import flask

#---------- MODEL IN MEMORY ----------------#

# Read the scientific data on breast cancer survival,
# Build a LogisticRegression predictor on it


#---------- URLS AND WEB PAGES -------------#

# Initialize the app
app = flask.Flask(__name__)

# Homepage
@app.route("/")
def viz_page():
    """
    Homepage: serve our visualization page, awesome.html
    """
    with open("awesome.html", 'r') as viz_file:
        return viz_file.read()

# Get an example and return it's score from the predictor model
@app.route("/score", methods=["POST"])
def score():
    """
    When A POST request with json data is made to this uri,
    Read the example from the json, predict probability and
    send it with a response
    """
    # Get decision score for our example that came with the request
    data = flask.request.json
#    x = np.matrix(data["example"])
    score = PREDICTOR.predict(x_h)[0]
    # Put the result in a nice dict so we can send it as json
    results = {"score": score}
    return flask.jsonify(results)

#--------- RUN WEB APP SERVER ------------#

# Start the app server on port 80
# (The default website port)
app.run(host='0.0.0.0')
app.run(debug=True)


OSError: [Errno 48] Address already in use

In [2138]:
def viz_page():
    """
    Homepage: serve our visualization page, awesome.html
    """
    with open("awesome.html", 'r') as viz_file:
        return viz_file.read()